In [1]:
import tools
import pandas as pd
import yfinance as yf

schema = "testuser"

In [2]:
con = tools.connect_postgres(abschnitt="azure_postgres")

In [3]:
t = yf.Ticker("TSLA")
tools.insert_earnings_quarter(con, t, schema)

In [4]:
df = t.history(start='2022-07-01')
df.head(3)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-06-30,224.509995,229.456665,218.863327,224.473328,94600500,0,0.0
2022-07-01,227.000000,230.229996,222.119995,227.263336,74460300,0,0.0
2022-07-05,223.000000,233.146667,216.166672,233.066666,84581100,0,0.0


In [5]:
df["symbol"] = t.info["symbol"]
del df["Dividends"]
del df["Stock Splits"]
df.head(3)

,open,high,low,close,volume,symbol
Date,,,,,,
2022-06-30,224.509995,229.456665,218.863327,224.473328,94600500,TSLA
2022-07-01,227.000000,230.229996,222.119995,227.263336,74460300,TSLA
2022-07-05,223.000000,233.146667,216.166672,233.066666,84581100,TSLA


In [11]:
df = df.reset_index()
df.columns = df.columns.str.lower()
df.head(3)

,date,open,high,low,close,volume,symbol
0,2022-06-30,224.509995,229.456665,218.863327,224.473328,94600500,TSLA
1,2022-07-01,227.000000,230.229996,222.119995,227.263336,74460300,TSLA
2,2022-07-05,223.000000,233.146667,216.166672,233.066666,84581100,TSLA


In [15]:
df.to_sql("history",con=con, schema="finance", if_exists="append", index=False)

57

In [11]:
def insert_history(con, ticker, start=None, schema="finance"):
    tools.insert_symbol(con, ticker, schema)
    df = ticker.history(start=start)
    df["symbol"] = ticker.info["symbol"]
    del df["Dividends"]
    del df["Stock Splits"]
    df = df.reset_index()
    df.columns = df.columns.str.lower()
    df.to_sql("history",con=con, schema=schema, if_exists="append", index=False)

In [7]:
#insert_history(con, t, None, "testuser")
#insert_history(con, t, None)
insert_history(con, t, schema="testuser")

In [ ]:
t = yf.Ticker("MSFT")
insert_history(con, t, schema="testuser")

In [19]:
query = f"""select max(date)+2 from testuser.history h 
    where symbol = 'MSFT'"""
max_datum = pd.read_sql(query, con).iloc[0,0]
print(max_datum)

2022-09-23


In [20]:
df = t.history(start=max_datum)
df

- MSFT: Invalid input - start date cannot be after end date. startDate = 1663884000, endDate = 1663837488


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [25]:
import datetime
print(max_datum)
print(datetime.date.today())
print(max_datum <= datetime.date.today())

2022-09-23
2022-09-22
False


In [24]:
if max_datum <= datetime.date.today():
    df = t.history(start=max_datum)
    print(df)

In [ ]:
def insert_history(con, ticker, start=None, schema="finance"):
    # Symbol in Tabelle symbol schreiben, wenn noch nicht vorhanden
    tools.insert_symbol(con, ticker, schema)
    # Bestimme das maximale Datum in Tabelle history in der DB
    query = f"""select max(date)+2 from testuser.history h 
        where symbol = 'MSFT'"""
    max_datum = pd.read_sql(query, con).iloc[0,0]
    # Wenn wir neue Werte haben
    if max_datum <= datetime.date.today():
        df = ticker.history(start=max_datum)
        df["symbol"] = ticker.info["symbol"]
        del df["Dividends"]
        del df["Stock Splits"]
        df = df.reset_index()
        df.columns = df.columns.str.lower()
        df.to_sql("history",con=con, schema=schema, if_exists="append", index=False)

In [28]:
df = t.history(start=max_datum)
df.shape[0]

- MSFT: Invalid input - start date cannot be after end date. startDate = 1663884000, endDate = 1663838192


0

In [30]:
def insert_history(con, ticker, start=None, schema="finance"):
    # Symbol in Tabelle symbol schreiben, wenn noch nicht vorhanden
    tools.insert_symbol(con, ticker, schema)
    symbol = ticker.info["symbol"]
    # Bestimme das maximale Datum in Tabelle history in der DB
    # +2, weil die history-Funktion die Daten von einem Tag vorher holt
    query = f"""select max(date)+2 from {schema}.history h 
        where symbol = '{symbol}'"""
    max_datum = pd.read_sql(query, con).iloc[0,0]
    df = ticker.history(start=max_datum)
    # Wenn die Zeilenanzahl größer als 0
    if df.shape[0] > 0:
        df["symbol"] = symbol
        del df["Dividends"]
        del df["Stock Splits"]
        df = df.reset_index()
        df.columns = df.columns.str.lower()
        df.to_sql("history",con=con, schema=schema, if_exists="append", index=False)

In [32]:
insert_history(con, t, schema="testuser")

Eintrag existiert schon
